In [1]:
# imports
import cv2 as cv
import numpy as np
import time
from matplotlib import pyplot as plt
import glob

# Time Measurement

In [2]:
time_measurements = {}

def start_time_measurement(eventName):
    if eventName not in time_measurements:
        time_measurements[eventName] = {
            "name": eventName,
            "start": [time.process_time_ns()],
            "end": []
        }
    elif len(time_measurements[eventName]["start"]) > len(time_measurements[eventName]["end"]):
        print(f"Time measure error: Event '{eventName}' not finished before reassignment!")
    else:
       time_measurements[eventName]["start"].append(time.process_time_ns()) 

def end_time_measurement(eventName):
    if eventName not in time_measurements:
        print(f"Time measure error: Event '{eventName}' not defined!")
    elif len(time_measurements[eventName]["end"]) >= len(time_measurements[eventName]["start"]):
        print(f"Time measure error: Event '{eventName}' not started before reassignment!")
    else:
       time_measurements[eventName]["end"].append(time.process_time_ns()) 

def analyse_time_measurements():
    for key, event in time_measurements.items():
        if len(event["start"]) != len(event["end"]):
            print(f"Time measure error: Event '{key}' has different amounts of values for start and end times!")
        else:
            event["avg"] = sum((event["end"][i] - event["start"][i]) for i in range(len(event["start"]))) / len(event["start"])

    # plt.boxplot([(i["end"] - i["start"])for i in list(time_measurements.values())])

# Kamerakalibrierung

In [3]:
# size of chessboard, minimum error with (7, 6), but there were severe artefacts at the borders (see error calculation at the end)
x, y = 9, 6

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((y * x, 3), np.float32)
objp[:, :2] = np.mgrid[0:x, 0:y].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.


images = glob.glob("./img/Udacity/calib/*.jpg")
for i, fname in enumerate(images):
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (x, y), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(
            gray, corners, (11, 11), (-1, -1), criteria
        )  # improve accuracy of corners
        imgpoints.append(corners)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))


def undistort_image(img):
    h, w = img.shape[:2]

    img_undist = cv.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    return img_undist[y : y + h, x : x + w]


# ~50% faster than undistort_image()
def undistort_image_remap(img):
    h, w = img.shape[:2]
    mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w, h), 5)
    dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
    # crop the image
    x, y, w, h = roi
    return dst[y : y + h, x : x + w]

# Perspektivtransformation

Mögliche Performance-verbesserung:

- Bild nach warp verkleinern, da ein großteil des Bildes aus wenigen Pixeln entsteht ->


In [4]:
# udacity images
src_udacity = np.float32([[191, 628], [531, 404], [1021, 628], [681, 404]])
dst_udacity = np.float32([[150, 720], [150, 10], [1000, 720], [1000, 10]])
M = cv.getPerspectiveTransform(src_udacity, dst_udacity)


def warp_image_udacity(img):
    image = cv.warpPerspective(img, M, (img.shape[1], img.shape[0]))
    image = cv.resize(
        image, (int(img.shape[1] / 2), int(img.shape[0] / 2))
    )  # resize to half size
    return image

# Sliding Windows

In [5]:
nwindows = 50
margin = 25
def sliding_windows(frame, window_width=200, minimum_whites=30, show_windows=False):
    # Histogram for image
    hist = np.sum(frame[frame.shape[0]//2:, :], axis=0)
        
    # Take peaks from left and right side of histogramm for starting points and add half margin
    mid = np.int(hist.shape[0] // 2)
    leftx_start = np.argmax(hist[:mid]) - window_width // 2
    rightx_start = np.argmax(hist[mid:]) + mid + window_width // 2
    # Window height based on number of windows
    window_height = np.int(frame.shape[0] // nwindows)
    
    # Calc points that are not zero in images
    nonzero = frame.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Initialize current positions for windows
    leftx_current = leftx_start
    rightx_current = rightx_start

    # Initialize values to be returned -> centers of windows
    lefts_good = np.empty(shape=(1,1), dtype=int)
    rights_good = np.empty(shape=(1,1), dtype=int)

    # Go through every window
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = frame.shape[0] - (window + 1) * window_height
        win_y_high = frame.shape[0] - window*window_height
        
        # Calculate boundaries of the window
        win_xleft_low = leftx_current - window_width  
        win_xleft_high = leftx_current + window_width  
        win_xright_low =  rightx_current - window_width 
        win_xright_high = rightx_current + window_width  
        
        # Identify the pixels that are not zero within window
        left_inds = ((nonzeroy >= win_y_low ) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        right_inds = ((nonzeroy >= win_y_low ) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # If more than minimum pixels are found -> recenter next window
        if len(left_inds) > minimum_whites:
            leftx_current = np.int(np.mean(nonzerox[left_inds]))
            lefts_good = np.append(lefts_good, leftx_current)
            if show_windows:
                cv.rectangle(frame, (leftx_current - margin, win_y_low),(leftx_current + margin, win_y_high),(255, 255, 255), 2)
        else:
            lefts_good = np.append(lefts_good, None)
        if len(right_inds) > minimum_whites:
            rightx_current = np.int(np.mean(nonzerox[right_inds]))
            rights_good = np.append(rights_good, rightx_current)
            if show_windows:
                cv.rectangle(frame, (rightx_current - margin, win_y_low),(rightx_current + margin, win_y_high),(255, 255, 255), 2)
        else:
            rights_good = np.append(rights_good, None)

    return mid, lefts_good, rights_good

# Main


In [6]:
# Open video file
video_file = "project"
capture = cv.VideoCapture("./img/Udacity/" + video_file + "_video.mp4")

# Check if camera opened successfully
if capture.isOpened() == False:
    print("Error opening video stream or file")

# Start timer for fps counter
start_timer = time.time() - 0.01
frame_count = -1

# Read every frame
while capture.isOpened():
    ret, frame = capture.read()

    # Check if there is another frame
    if frame is None:
        break
    orig = frame.copy()

    # Calculate Frame rate
    frame_count += 1
    ellapsed_time = time.time() - start_timer
    frame_rate = frame_count / ellapsed_time

    if ret == True:

        frame = undistort_image_remap(frame)
        frame = warp_image_udacity(frame)

        #---------- Masking ----------
        ## convert to hsv
        hls_frame = cv.cvtColor(frame, cv.COLOR_BGR2HLS)

        ## mask for white
        white_mask = cv.inRange(hls_frame, (0, 200, 0), (255, 255,255))

        ## mask for yellow
        # yellow_mask = cv.inRange(hls_frame, (20,90,200), (26, 255, 255))
        yellow_mask = cv.inRange(hls_frame, (10,0,100), (40, 255, 255))

        ## final mask and masked
        mask = cv.bitwise_or(white_mask, yellow_mask)
        frame = cv.bitwise_and(frame,frame, mask=mask)

        #---------- Filtering ----------
        greyscale_frame = cv.cvtColor(frame, cv.COLOR_HLS2BGR)
        greyscale_frame = cv.cvtColor(greyscale_frame, cv.COLOR_BGR2GRAY)

        # deblur image
        # frame = cv.medianBlur(greyscale_frame,5)
        greyscale_frame = cv.Canny(greyscale_frame, 50, 150)

        #---------- Sliding Windows ----------
        # midpoint, lefts, rights = sliding_windows(frame, minimum_whites=margin, show_windows=True)
        # window_height = np.int(frame.shape[0] // nwindows)
        # for window in range(nwindows):
        #     win_y_low = frame.shape[0] - (window + 1) * window_height
        #     win_y_high = frame.shape[0] - window*window_height
        #     if lefts[window]:
        #         cv.rectangle(frame, (lefts[window] - margin, win_y_low),(lefts[window] + margin, win_y_high),(255, 255, 255), 2)
        #     if rights[window]:
        #         cv.rectangle(frame, (rights[window] - margin, win_y_low),(rights[window] + margin, win_y_high),(255, 255, 255), 2)

        # y = np.linspace(len(frame), 0, nwindows+1).astype(int)
        # left_line = np.array(list(zip(lefts[1:], y)))
        # right_line = np.array(list(zip(rights[1:], y)))

        # cv.polylines(frame, [left_line], 0, (255,255,255), 2)
        # cv.polylines(frame, [right_line], 0, (255,255,255), 2)
        #---------------------------------------------------------------------------------------------------------------------------------------------------------------
        # #treshhold for image
        # thresh = 160
        # ret, frame = cv.threshold(frame, thresh=thresh, maxval=255, type=cv.THRESH_BINARY)

        #deblur image
        # frame = cv.medianBlur(frame,5)

        # # Schärfungsfilter
        # kernel2 = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]],np.float32)
        # frame = cv.filter2D(frame,-1,kernel2)

        # wende Blurring an
        # frame = cv.GaussianBlur(frame, (5, 5), cv.BORDER_DEFAULT)

        # wende Canny Edge an
        # frame = cv.Canny(frame, 50, 80)

        # # Wende mophologische Filter(Closing und Opening) an
        # small_kernel = np.array([[0,1,0],[1,1,1],[0,1,0]], 'uint8')
        # frame = cv.morphologyEx(frame, cv.MORPH_CLOSE, small_kernel, iterations=20)

        # frame = cv.morphologyEx(frame, cv.MORPH_OPEN, small_kernel, iterations=2)

        # Add frame rate to video
        cv.putText(greyscale_frame, "FPS: " + str(round(frame_rate)), (0, 25),
                   cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        cv.imshow("Frame", greyscale_frame)

        # Close video with letter 'q'
        if cv.waitKey(15) & 0xFF == ord("q"):
            break
    else:
        break

# When everything done, release the video capture object
capture.release()
cv.destroyAllWindows()